In [3]:
import pandas as pd
import numpy as np
import re
import pickle

In [ ]:
import pyspark as ps

spark = ps.sql.SparkSession.builder \
            .master("local[4]").getOrCreate()

### Cleaning

In [2]:
p13 = pd.read_csv('/Volumes/Seagate/Galvanize/nj_payments_2013.csv',dtype={'zip':object})
p13.drop(['Unnamed: 0'],axis=1, inplace=True)

In [8]:
p13.head(2)

,id,fn,mn,ln,address1,state,zip,specialty,amount
0,41273.0,MANU,V,CHAKRAVARTHY,1 RWJ PLACE,NJ,08901,Allopathic & Osteopathic Physicians|Internal M...,80.5
1,41273.0,MANU,V,CHAKRAVARTHY,1 RWJ PLACE,NJ,08901,Allopathic & Osteopathic Physicians|Internal M...,58.0


In [30]:
p13['ln'] = p13['ln'].map(lambda x: x.replace(' ',''))
p13['fn'] = p13['fn'].map(lambda x: x.replace(' ',''))
p13['ln'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p13['ln']]
p13['fn'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p13['fn']]
p13['address1'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p13['address1']]

In [31]:
p13['zip'] = [x.replace('-','') for x in p13['zip']]
p13['zip'] = [('0'+str(x)) if len(str(x))==8 or len(str(x))==4 else str(x) for x in p13['zip']]

In [32]:
p13.head(5)

,id,fn,mn,ln,address1,state,zip,specialty,amount
0,41273.0,MANU,V,CHAKRAVARTHY,1 RWJ PLACE,NJ,08901,Allopathic & Osteopathic Physicians|Internal M...,80.50
1,41273.0,MANU,V,CHAKRAVARTHY,1 RWJ PLACE,NJ,08901,Allopathic & Osteopathic Physicians|Internal M...,58.00
2,41273.0,MANU,V,CHAKRAVARTHY,1 RWJ PLACE,NJ,08901,Allopathic & Osteopathic Physicians|Internal M...,62.77
3,41273.0,MANU,V,CHAKRAVARTHY,1 RWJ PLACE,NJ,08901,Allopathic & Osteopathic Physicians|Internal M...,2.99
4,41273.0,MANU,V,CHAKRAVARTHY,1 RWJ PLACE,NJ,08901,Allopathic & Osteopathic Physicians|Internal M...,66.00


In [33]:
p13.to_csv('/Volumes/Seagate/Galvanize/nj_payments_2013.csv')

### OG work

In [3]:
p13 = pd.read_csv('/Volumes/Seagate/Galvanize/nj_payments_2013.csv',dtype={'zip':object})
p13.drop(['Unnamed: 0'],axis=1, inplace=True)
p13.head(2)

,id,fn,mn,ln,address1,state,zip,specialty,amount
0,41273.0,MANU,V,CHAKRAVARTHY,1 RWJ PLACE,NJ,08901,Allopathic & Osteopathic Physicians|Internal M...,80.5
1,41273.0,MANU,V,CHAKRAVARTHY,1 RWJ PLACE,NJ,08901,Allopathic & Osteopathic Physicians|Internal M...,58.0


In [29]:
d = p13.groupby(['specialty','ln','fn','id',]).count()['amount']

In [31]:
d.to_frame()

amount
specialty                                          ln                fn           id               
Agencies|Community/Behavioral Health               BLACK             EDWARD       23419.0         7
Allopathic & Osteopathic Physicians|Allergy & I... AMARA             SREENIVASRAO 341964.0        1
                                                   AXELROD MALAGOLD  SARA         322803.0        3
                                                   AXELROD-MALAGOLD  SARA         322803.0        1
                                                   BANTZ             ERIC         1220958.0       2
                                                   BARISCIANO        LISA         147425.0        5
                                                   BELECANECH        GEORGE       198712.0       11
                                                   BERLIN            PAUL         288160.0       10
                                                   BHAMBRI           NEHA         79685.0         3
                                                   BHASIN            YASMIN       233339.0        4
                                                   BIGELSEN          STEPHEN      172046.0        2
                                                   BLUME             JESSICA      150130.0        2
                                                   BOKHARI           TAHIRA       835809.0        2
                                                   BOSSO             JOHN         103219.0        2
                                                   BROWN             DAVID        116738.0        3
                                                   CAMACHO-HALILI    MARIE        274386.0       10
                                                   CAPITLE           EUGENIO      116449.0        2
                                                   CASE              PHILIP       431957.0        2
                                                   CAUCINO           JULIE        16830.0         8
                                                   CENTENO           LIGAYA       202118.0       13
                                                   CHANG             CINDY        154085.0        5
                                                   COLENDA           MARYANN      254659.0        8
                                                                     MARYANNE     254659.0       38
                                                   DADHANIA          MAHENDRA     885345.0        1
                                                   DANGELO           SALVATORE    10539.0         1
                                                   DE LA CRUZ        ANTONIO      292929.0        2
                                                   DECOTIIS          BRUCE        306369.0       12
                                                   DEFUSCO           CARMINE      224099.0       10
                                                   DRUCE             HOWARD       312466.0        2
                                                   DVORIN            DONALD       150758.0        6
...                                                                                             ...
Student, Health Care|Student in an Organized He... THOSANI           MAYA         341189.0        2
                                                   TIRUVURY          HEMAVARNA    1304006.0       1
                                                   TONG              VANESSA      103330.0        3
                                                   TORBUS            ANDRZEJ      667821.0        2
                                                   TOVMASIAN         LUCY         166607.0        3
                                                   TURKISH           JENNIFER     126959.0       11
                                                   VADHAVKAR         AARTI        181559.0        1
                                                   VAKAR             EMIL         1300085.0  

In [23]:
d2 = p13.groupby(['id','fn','ln','mn','zip','specialty','address1']).sum()['amount']

In [24]:
d2 = d2.to_frame()
d2.head(5)

amount
id    fn        ln       mn  zip       specialty                                          address1                                    
11.0  ALEXANDER GOTESMAN NAN 08690     Allopathic & Osteopathic Physicians|Urology        1374 WHITEHORSE HAMILTON SQUARE ROAD   28.23
75.0  ELISABETH ANACIUS  NAN 088224603 Allopathic & Osteopathic Physicians|Family Medi... 2100 WESCOTT DR                       124.36
86.0  ANDREW    NAJOVITS J   07450     Allopathic & Osteopathic Physicians|Internal Me... 104 CHESTNUT ST                        14.26
                         NAN 07450     Allopathic & Osteopathic Physicians|Internal Me... 104 CHESTNUT STREET 3RD FLOOR          29.35
144.0 MICHAEL   SUBIK    NAN 07071     Allopathic & Osteopathic Physicians|Internal Me... 160 RIDGE ROAD                         92.60

In [7]:
d2.reset_index(inplace=True)

In [8]:
d2.specialty.unique()

array(['Allopathic & Osteopathic Physicians|Urology',
       'Allopathic & Osteopathic Physicians|Family Medicine',
       'Allopathic & Osteopathic Physicians|Internal Medicine|Cardiovascular Disease',
       'Allopathic & Osteopathic Physicians|Internal Medicine',
       'Podiatric Medicine & Surgery Service Providers|Podiatrist',
       'Podiatric Medicine & Surgery Service Providers|Podiatrist|Foot & Ankle Surgery',
       'Allopathic & Osteopathic Physicians|Ophthalmology',
       'Allopathic & Osteopathic Physicians|Emergency Medicine',
       'Allopathic & Osteopathic Physicians|Pediatrics',
       'Allopathic & Osteopathic Physicians|Internal Medicine|Rheumatology',
       'Dental Providers|Dentist',
       'Allopathic & Osteopathic Physicians|Obstetrics & Gynecology|Reproductive Endocrinology',
       'Dental Providers|Dentist|General Practice',
       'Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Psychiatry',
       'Allopathic & Osteopathic Physicians|Pathology

In [9]:
payids = d2.copy()

In [12]:
payids['zip5'] = payids['zip'].map(lambda x: x[:5])

In [25]:
payids.drop('amount',axis=1,inplace=True)

In [94]:
payids['mn'] = payids['mn'].map(lambda x: str(x).replace(' ',''))
payids['mn'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in payids['mn']]

In [95]:
payids.isnull().sum()

id           0
fn           0
ln           0
mn           0
zip          0
specialty    0
address1     0
zip5         0
dtype: int64

In [47]:
payids.replace('NAN', np.NaN,inplace=True)

In [49]:
mis = payids[payids['id'].isin(payids['id'][payids['id'].duplicated()])].sort_values("id")

In [50]:
payids.to_csv('/Volumes/Seagate/Galvanize/nj_13_payment_ids.csv')
mis.to_csv('/Volumes/Seagate/Galvanize/nj_13_multi_name_payment_ids.csv')

# <font color='red'>START HERE</font>

In [51]:
payids = pd.read_csv('/Volumes/Seagate/Galvanize/nj_13_payment_ids.csv',dtype={'zip':object, 'zip5':object})
mis = pd.read_csv('/Volumes/Seagate/Galvanize/nj_13_multi_name_payment_ids.csv',dtype={'zip':object, 'zip5':object})

In [52]:
payids.drop('Unnamed: 0',axis=1, inplace=True)
mis.drop('Unnamed: 0',axis=1, inplace=True)

In [53]:
mis.head()

,id,fn,ln,mn,zip,specialty,address1,zip5
0,86.0,ANDREW,NAJOVITS,J,07450,Allopathic & Osteopathic Physicians|Internal M...,104 CHESTNUT ST,07450
1,86.0,ANDREW,NAJOVITS,NaN,07450,Allopathic & Osteopathic Physicians|Internal M...,104 CHESTNUT STREET 3RD FLOOR,07450
2,144.0,MICHAEL,SUBIK,NaN,07071,Allopathic & Osteopathic Physicians|Internal M...,160 RIDGE ROAD,07071
3,144.0,MICHAEL,SUBIK,NaN,07071,Podiatric Medicine & Surgery Service Providers...,160 RIDGE RD,07071
4,144.0,MICHAEL,SUBIK,NaN,07071,Podiatric Medicine & Surgery Service Providers...,160 RIDGE RD,07071


In [54]:
payids.head()

,id,fn,ln,mn,zip,specialty,address1,zip5
0,11.0,ALEXANDER,GOTESMAN,NaN,08690,Allopathic & Osteopathic Physicians|Urology,1374 WHITEHORSE HAMILTON SQUARE ROAD,08690
1,75.0,ELISABETH,ANACIUS,NaN,088224603,Allopathic & Osteopathic Physicians|Family Med...,2100 WESCOTT DR,08822
2,86.0,ANDREW,NAJOVITS,J,07450,Allopathic & Osteopathic Physicians|Internal M...,104 CHESTNUT ST,07450
3,86.0,ANDREW,NAJOVITS,NaN,07450,Allopathic & Osteopathic Physicians|Internal M...,104 CHESTNUT STREET 3RD FLOOR,07450
4,144.0,MICHAEL,SUBIK,NaN,07071,Allopathic & Osteopathic Physicians|Internal M...,160 RIDGE ROAD,07071


In [3]:
print("# Unique IDs: {}".format(len(p13.groupby(['id']).count()['amount'])))

# Unique IDs: 18828


In [16]:
print("Amount mismatched payments = {}".format(sum(d2[d2['id'].isin(d2['id'][d2['id'].duplicated()])].sort_values("id")['amount'])))
print('Mismatched unique IDs = {}'.format(len(d2[d2['id'].isin(d2['id'][d2['id'].duplicated()])].sort_values("id")['id'].unique())))

Amount mismatched payments = 4550
Mismatched unique IDs = 255


In [171]:
#This function doesn't include middle name because it was giving me toruble
d = {}
def payment_id(x):
    if x['id'] in d:
        d[x['id']]['fn'].add(x['fn'])
        d[x['id']]['ln'].add(x['ln'])
        d[x['id']]['zip5'].add(x['zip5'])
        d[x['id']]['address1'].add(x['address1'])
        d[x['id']]['specialty'].update(list(x['specialty'].split('|')))
    elif x['id'] not in d:
        d[x['id']] = {'fn':set([x['fn']]), \
                     'ln':set([x['ln']]), \
                     'zip5':set([x['zip5']]), \
                     'address1':set([x['address1']]), \
                     'specialty':set(list(x['specialty'].split('|')))}

In [56]:
"""d = {}
def f(x):
    if x['id'] in d:
        d[x['id']]['fn'].add(x['fn'])
        d[x['id']]['mn'].add(x['mn'])
        d[x['id']]['ln'].add(x['ln'])
        d[x['id']]['zip5'].add(x['zip5'])
        d[x['id']]['address1'].add(x['address1'])
        d[x['id']]['specialty'].add(x['specialty'])
    elif x['id'] not in d:
        d[x['id']] = {'fn':set([x['fn']]), \
                     'mn':set([x['mn']]), \
                     'ln':set([x['ln']]), \
                     'zip5':set([x['zip5']]), \
                     'address1':set([x['address1']]), \
                     'specialty':set([x['specialty']])}"""

In [172]:
payids.apply(payment_id,axis=1)
len(d)

18767

In [175]:
pay_id_dict = d
len(pay_id_dict)

18767

In [176]:
pickle.dump(pay_id_dict, open('13pay13dict.pkl', 'wb'))

In [163]:
#How to check if values are equal

print(d[11.0]['specialty'].intersection(npi_dict[1003002072]['specialty']),npi_dict[1003002072]['specialty'].intersection(d[11.0]['specialty']))

{'Urology'} {'Urology'}


In [177]:
n=0
for i in pay_id_dict[11.0]:
    if len(pay_id_dict[11.0][i].intersection(npi_dict[1003002072][i])) > 0:
        n+=1
if n >= 4:
    print('llit')

llit


In [3]:
npi_dict = pickle.load(open('13npidict.pkl','rb'))

In [4]:
pay_id_dict = pickle.load(open('13pay13dict.pkl','rb'))

**Making a copy dict so i dont fuck myself over**

In [5]:
n = npi_dict 
d = pay_id_dict

In [17]:
len(n),len(d)

(21710, 18767)

In [7]:
npi_pay_dict = {}

In [27]:
def combine_npi_payid(d,n):
    """
    Input:
        d - Dict of pay_ids as keys with dict as value containing address, fn, ln, specialty, zip5 as keys 
                & all possible alts. for that pay_id as values
        n - Dict of npis as keys with dict as value containing address, fn, ln, specialty, zip5 as keys 
                & all possible alts. for that npi as values 
    Output: 
        None
    """
    npi_pay_dict = {}
    for pay_id in d:
        for npi in n:
            t_count = 0
            for val in d[pay_id]:
                if len(d[pay_id][val].intersection(n[npi][val])) >0:
                    t_count += 1
            if t_count >= 4:
                npi_pay_dict[pay_id] = npi

<font color='red'>Saving the dict as a pickle so I don't have to run this again</font> 

In [31]:
pickle.dump(npi_pay_dict, open('13_linked.pkl', 'wb'))

In [20]:
combine_npi_payid(d,n)

In [11]:
len(npi_pay_dict)

10345

In [13]:
c = {k:v for k,v in d.items() if k not in npi_pay_dict}
#All the open payment IDs that weren't transfered

In [23]:
18767-10345

8422

In [14]:
c

{86.0: {'address1': {'104 CHESTNUT ST', '104 CHESTNUT STREET 3RD FLOOR'},
  'fn': {'ANDREW'},
  'ln': {'NAJOVITS'},
  'specialty': {'Allopathic & Osteopathic Physicians',
   'Cardiovascular Disease',
   'Internal Medicine'},
  'zip5': {'07450'}},
 242.0: {'address1': {'185 BRIDGE PLZ N'},
  'fn': {'EDITH'},
  'ln': {'SZABO'},
  'specialty': {'Allopathic & Osteopathic Physicians', 'Emergency Medicine'},
  'zip5': {'07024'}},
 244.0: {'address1': {'595 CHESTNUT RIDGE RD'},
  'fn': {'ADAM'},
  'ln': {'BLOOMFIELD'},
  'specialty': {'Allopathic & Osteopathic Physicians', 'Pediatrics'},
  'zip5': {'07677'}},
 415.0: {'address1': {'140 ALLEN RD'},
  'fn': {'MARCY'},
  'ln': {'MAGUIRE'},
  'specialty': {'Allopathic & Osteopathic Physicians',
   'Obstetrics & Gynecology',
   'Reproductive Endocrinology'},
  'zip5': {'07920'}},
 425.0: {'address1': {'224 ROUTE 37 E'},
  'fn': {'THOMAS'},
  'ln': {'HADA'},
  'specialty': {'Dental Providers', 'Dentist', 'General Practice'},
  'zip5': {'08753'}},
 

In [21]:
len(c)

8422

### <font color='red'>pay_id_dict</font> = payment ids & all the names & shit

### <font color='blue'>npi_dict</font> = NPI & all the names & shit

In [160]:
j=0
for i in d:
    if j < 2:
        j+=2
        print(i,d[i])

11.0 {'fn': {'ALEXANDER'}, 'ln': {'GOTESMAN'}, 'zip5': {'08690'}, 'address1': {'1374 WHITEHORSE HAMILTON SQUARE ROAD'}, 'specialty': {'Urology', 'Allopathic & Osteopathic Physicians|Urology', 'Allopathic & Osteopathic Physicians'}}


In [162]:
npi_dict[1003002072]['specialty'], d[11.0]['specialty']

({'Urology'},
 {'Allopathic & Osteopathic Physicians',
  'Allopathic & Osteopathic Physicians|Urology',
  'Urology'})

In [139]:
d[11.0]['specialty']

{'Allopathic & Osteopathic Physicians|Urology'}

In [144]:
list(npi_dict[1003002072]['specialty'])[0] in list(d[11.0]['specialty'])[0]

True

In [159]:
for i in d:
    if 'HARISH' in d[i]['fn']:
        print(i, d[i])

85566.0 {'fn': {'HARISH'}, 'ln': {'KAKKILAYA'}, 'zip5': {'08079'}, 'address1': {'8 BY PASS RD'}, 'specialty': {'Surgery', 'Allopathic & Osteopathic Physicians', 'Allopathic & Osteopathic Physicians|Surgery'}}
118236.0 {'fn': {'HARISH'}, 'ln': {'SHAH'}, 'zip5': {'07470'}, 'address1': {'468 PARISH DR', '468 PARISH DR STE 4'}, 'specialty': {'Internal Medicine', 'Allopathic & Osteopathic Physicians|Internal Medicine|Hematology & Oncology', 'Allopathic & Osteopathic Physicians', 'Hematology & Oncology'}}
138060.0 {'fn': {'HARISH'}, 'ln': {'IYER'}, 'zip5': {'08540'}, 'address1': {'13314 WINTERBERRY WAY'}, 'specialty': {'Internal Medicine', 'Allopathic & Osteopathic Physicians', 'Allopathic & Osteopathic Physicians|Internal Medicine'}}
239819.0 {'fn': {'HARISH'}, 'ln': {'MALHOTRA'}, 'zip5': {'07901'}, 'address1': {'33 OVERLOOK RD'}, 'specialty': {'Psychiatry', 'Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Psychiatry', 'Allopathic & Osteopathic Physicians', 'Psychiatry & Neurolog

In [161]:
j=0
for i in npi_dict:
    if j < 2:
        j+=2
        print(i,npi_dict[i])

1003002072 {'fn': {'ALEXANDER'}, 'ln': {'GOTESMAN'}, 'zip5': {'08690'}, 'address1': {'1374 WHITEHORSE HAMILTON SQUARE RD'}, 'specialty': {'Urology'}}


In [63]:
#This is the dict where i included MN
j=0
for i in npi_d:
    if j < 2:
        j+=2
        print(i,npi_d[i])

1003002072 {'fn': {'ALEXANDER'}, 'mn': {nan}, 'ln': {'GOTESMAN'}, 'zip5': {'08690'}, 'address1': {'1374 WHITEHORSE HAMILTON SQUARE RD'}, 'specialty': {'Urology'}}


In [127]:
print(list(npi_d[1003002072]['mn'])[0],np.isnan(list(npi_d[1003002072]['mn'])[0]))

nan True


# Doing same stuff on the 2014 data

In [79]:
#Then I pick only the columns I want & load the entire csv
pay14 = pd.read_csv('/Volumes/Seagate/Galvanize/2014 Open Payments/OP_DTL_GNRL_PGYR2014_P01172018.csv', \
                    usecols=['Physician_Profile_ID','Physician_First_Name','Physician_Middle_Name', \
                             'Physician_Last_Name','Recipient_State','Total_Amount_of_Payment_USDollars', \
                            'Recipient_Primary_Business_Street_Address_Line1', 'Physician_Specialty', 'Recipient_Zip_Code'])
#Grabbing only the NJ practioners
p13 = pay14[pay14['Recipient_State'].isin(['NJ'])]
p13.to_csv('/Volumes/Seagate/Galvanize/nj_payments_2014.csv')

/Users/kunal/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [80]:
p13 = pd.read_csv('/Volumes/Seagate/Galvanize/nj_payments_2014.csv',dtype={'Recipient_Zip_Code':object})

In [81]:
p13.drop('Unnamed: 0',axis=1,inplace=True)

In [82]:
p13.columns

Index(['Physician_Profile_ID', 'Physician_First_Name', 'Physician_Middle_Name',
       'Physician_Last_Name',
       'Recipient_Primary_Business_Street_Address_Line1', 'Recipient_State',
       'Recipient_Zip_Code', 'Physician_Specialty',
       'Total_Amount_of_Payment_USDollars'],
      dtype='object')

In [83]:
l = p13.columns
p13.rename(columns={l[0]:'id',l[1]:'fn',l[2]:'mn',l[3]:'ln',l[4]:'address1',l[5]:'state',l[6]:'zip',l[-2]:'specialty',l[-1]:'amount'},inplace=True)
#Cleaning the first & last name (removing space & symbols) and the address (removing any symbols)
p13['ln'] = p13['ln'].map(lambda x: str(x).replace(' ',''))
p13['fn'] = p13['fn'].map(lambda x: str(x).replace(' ',''))
p13['mn'] = p13['mn'].map(lambda x: str(x).replace(' ',''))
p13['ln'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p13['ln']]
p13['fn'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p13['fn']]
p13['mn'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p13['mn']]
p13['address1'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p13['address1']]
#Cleaning the zip codes (removing dashes & making sure they are not missing the leading 0 that most NJ zip codes have)
p13['zip'] = [x.replace('-','') for x in p13['zip']]
p13['zip'] = [('0'+str(x)) if len(str(x))==8 or len(str(x))==4 else str(x) for x in p13['zip']]
#Making a column for the 5 digit zip
p13['zip5'] = p13['zip'].map(lambda x: x[:5])

In [84]:
payids = p13.groupby(['id','fn','ln','mn','zip5','specialty','address1']).count()['amount']

In [85]:
payids = payids.to_frame()

In [86]:
payids.reset_index(inplace=True)

In [87]:
#Dropping amount column because it's not necessary rn
payids.drop('amount',axis=1,inplace=True)
#Finding all the mispelled names/addresses/etc.
mis = payids[payids['id'].isin(payids['id'][payids['id'].duplicated()])].sort_values("id")
#Saving as a CSV for future use
payids.to_csv('/Volumes/Seagate/Galvanize/nj_14_payment_ids.csv')
mis.to_csv('/Volumes/Seagate/Galvanize/nj_14_multi_name_payment_ids.csv')

In [63]:
#This function doesn't include middle name because it was giving me toruble
d = {}
def payment_id(x):
    if x['id'] in d:
        d[x['id']]['fn'].add(x['fn'])
        d[x['id']]['ln'].add(x['ln'])
        d[x['id']]['zip5'].add(x['zip5'])
        d[x['id']]['address1'].add(x['address1'])
        d[x['id']]['specialty'].update(list(x['specialty'].split('|')))
    elif x['id'] not in d:
        d[x['id']] = {'fn':set([x['fn']]), \
                     'ln':set([x['ln']]), \
                     'zip5':set([x['zip5']]), \
                     'address1':set([x['address1']]), \
                     'specialty':set(list(x['specialty'].split('|')))}

In [88]:
payids.head()

,id,fn,ln,mn,zip5,specialty,address1
0,11.0,ALEXANDER,GOTESMAN,NAN,08690,Allopathic & Osteopathic Physicians|Urology,1374 WHITEHORSE HAMILTON SQUARE RD
1,11.0,ALEXANDER,GOTESMAN,NAN,08690,Allopathic & Osteopathic Physicians|Urology,1374 WHITEHRSE HAM RD STE 202
2,86.0,ANDREW,NAJOVITS,J,07430,Allopathic & Osteopathic Physicians|Internal M...,10 FRANKLIN TPKE
3,86.0,ANDREW,NAJOVITS,JOSEPH,07450,Allopathic & Osteopathic Physicians|Internal M...,104 CHESTNUT ST
4,86.0,ANDREW,NAJOVITS,NAN,07430,Allopathic & Osteopathic Physicians|Internal M...,10 FRANKLIN TPKE


In [64]:
payids.apply(payment_id,axis=1)
len(d)

24267

In [42]:
pickle.dump(d, open('14pay14dict.pkl', 'wb'))

In [26]:
#First I load only a small subset of the data to get an idea of the columns I'll need to use for joining
pres14 = pd.read_csv('/Volumes/Seagate/Galvanize/Prescriptions 2014.csv', nrows=100)
#Then I pick only the columns I want & load the entire csv
scripts14 = pd.read_csv('/Volumes/Seagate/Galvanize/Prescriptions 2014.csv',usecols=list(pres14.columns[:6]))
#Grabbing only the NJ practioners
scripts14nj = scripts14[scripts14['nppes_provider_state'].isin(['NJ'])]
#Renaming the columns so they're easier to call
l = scripts14nj.columns
scripts14nj.rename(columns={l[0]:'npi',l[1]:'ln',l[2]:'fn',l[3]:'city',l[4]:'state',l[5]:'specialty'},inplace=True)

/Users/kunal/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [27]:
#Upper casing the first name, last name, and city
scripts14nj['ln'] = [x.upper() for x in scripts14nj['ln']]
scripts14nj['fn'] = [str(x).upper() for x in scripts14nj['fn']]
scripts14nj['city'] = [x.upper() for x in scripts14nj['city']]
scripts14nj.to_csv('/Volumes/Seagate/Galvanize/2014_scriptsnj.csv')

/Users/kunal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kunal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/kunal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [45]:
#Grouping so I don't have a bunch of duplicate NPIs & I can look for mispellings
s14 = scripts14nj.groupby(['npi','ln','fn','city','specialty']).count()['state'].to_frame()
#Resetting the index becuase all my groupbys are now the index, which wont work for a merge
s14.reset_index(inplace=True)
#Checking if any of the people's names are misspelled
dup14 = s14[s14['npi'].isin(s14['npi'][s14['npi'].duplicated()])].sort_values("npi")

In [46]:
dup14

,npi,ln,fn,city,specialty,state


In [47]:
npi = pd.read_csv('/Volumes/Seagate/Galvanize/nj_idv_npi_may.csv',dtype={'zip':object})

In [49]:
npi.drop('Unnamed: 0',axis=1,inplace=True)
npi.head()

,NPI,Entity Type Code,ln,fn,mn,address1,state,zip
0,1477556421,1.0,ROTH,STEVEN,S.,1107 MANTUA PIKE,NJ,080511606
1,1508869553,1.0,VAN HOUTEN-SAUTER,LEE,ANN,220 PINE ST,NJ,080941137
2,1285637223,1.0,LOWRY,STEVEN,MICHAEL,220 PINE ST,NJ,080941137
3,1225031370,1.0,SY,NENA,L,350 BOULEVARD,NJ,070552840
4,1376546325,1.0,GIULIANO,MICHAEL,GERARD,622 FRANKLIN AVE,NJ,071101247


In [51]:
len(npi), len(npi.loc[npi['Entity Type Code']==1.0])

(102889, 102889)

In [52]:
npi.drop('Entity Type Code',axis=1,inplace=True)

In [54]:
script_npi = s14.merge(npi, left_on='npi',right_on='NPI')
#Dropping unecessary column & the NPI column because it's repeated
script_npi.drop(['NPI','state_x','state_y'],axis=1,inplace=True)

In [55]:
#Cleaning the joined dataframe, removing essentially all punctuation & spaces from the first or last name
script_npi['ln_x'] = script_npi['ln_x'].map(lambda x: x.replace(' ',''))
script_npi['ln_y'] = script_npi['ln_y'].map(lambda x: x.replace(' ',''))
script_npi['fn_x'] = script_npi['fn_x'].map(lambda x: x.replace(' ',''))
script_npi['fn_y'] = script_npi['fn_y'].map(lambda x: x.replace(' ',''))
script_npi['mn'] = script_npi['mn'].map(lambda x: str(x).replace(' ',''))
script_npi['ln_x'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['ln_x']]
script_npi['ln_y'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['ln_y']]
script_npi['fn_x'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['fn_x']]
script_npi['fn_y'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['fn_y']]
script_npi['mn'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['mn']]
script_npi['address1'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['address1']]
#Cleaning the zip codes, NJ has a leading 0 in the zip so it might of been dropped
script_npi['zip'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['zip']]
script_npi['zip'] = [('0'+str(x)) if len(str(x))==8 or len(str(x))==4 else str(x) for x in script_npi['zip']]

In [56]:
#Making a column for the 5 digit zip
script_npi['zip5'] = script_npi['zip'].map(lambda x: x[:5])
"Dealing with mismatched/mispelled names"
#This just lets me see names where the first name & last name don't match up
script_npi.loc[(script_npi['ln_x']!=script_npi['ln_y']) & (script_npi['fn_x']!=script_npi['fn_y'])]

,npi,ln_x,fn_x,city,specialty,ln_y,fn_y,mn,address1,zip,zip5


In [57]:
mis_name = script_npi.loc[(script_npi['ln_x']!=script_npi['ln_y']) | (script_npi['fn_x']!=script_npi['fn_y'])]
mis_name.head(5)

,npi,ln_x,fn_x,city,specialty,ln_y,fn_y,mn,address1,zip,zip5
26,1003076589,PISANO,DOMINICK,TOMS RIVER,Dentist,PISANO,DOMINIC,NAN,70 LACEY RD,087592931,08759
343,1013221050,GEDAKA,GERALYN,CHERRY HILL,Nurse Practitioner,SULLIVAN,GERALYN,M,401 KINGS HWY S,080342500,08034
767,1033219142,MELTON,LAURA,BELVIDERE,Nurse Practitioner,BAYLOR,LAURA,M,540 COUNTY ROAD 519,078232672,07823
1054,1043453616,MANSSON,SARAHJANE,BROWNS MILLS,Internal Medicine,MANSSON,SARAH,NAN,1200 E RIDGEWOOD AVE,074503957,07450
1280,1053528356,RICHARDSON,SONYA,EAST ORANGE,Nurse Practitioner,GOODRICH,SONYA,L,1200 CLINTON AVE,071112070,07111


In [58]:
#Saving these as CSVs for future use
script_npi.to_csv('/Volumes/Seagate/Galvanize/nj_14_scrip_npi.csv')
mis_name.to_csv('/Volumes/Seagate/Galvanize/nj_14_multi_name_npi.csv')

In [65]:
#This function doesn't include middle name because it was giving me toruble
n = {}
def script_npi_dict(x):
    if x['npi'] in n:
        n[x['npi']]['fn'].update([x['fn_x'],x['fn_y']])
        n[x['npi']]['ln'].update([x['ln_y'],x['ln_y']])
        n[x['npi']]['zip5'].add(x['zip5'])
        n[x['npi']]['address1'].add(x['address1'])
        n[x['npi']]['specialty'].add(x['specialty'])
    elif x['npi'] not in n:
        n[x['npi']] = {'fn':set([x['fn_x'],x['fn_y']]), \
                     'ln':set([x['ln_x'],x['ln_y']]), \
                     'zip5':set([x['zip5']]), \
                     'address1':set([x['address1']]), \
                     'specialty':set([x['specialty']])}

In [62]:
script_npi.head()

,npi,ln_x,fn_x,city,specialty,ln_y,fn_y,mn,address1,zip,zip5
0,1003001678,EIRICH,REBECCA,TURNERSVILLE,Nurse Practitioner,EIRICH,REBECCA,NAN,188 FRIES MILL RD,080122015,08012
1,1003002072,GOTESMAN,ALEXANDER,HAMILTON,Urology,GOTESMAN,ALEXANDER,NAN,1374 WHITEHORSE HAMILTON SQUARE RD,086903701,08690
2,1003002858,GIBSON,AMY,PLAINFIELD,Nurse Practitioner,GIBSON,AMY,R,1314 PARK AVE,070603253,07060
3,1003011891,SPIRA,ETAN,BELLEVILLE,Gastroenterology,SPIRA,ETAN,B,5 FRANKLIN AVE,071093532,07109
4,1003012410,TINDOC,LORELANE,CLIFTON,Family Practice,TINDOC,LORELANE,PAGULAYAN,780 ALLWOOD RD,070121923,07012


In [66]:
script_npi.apply(script_npi_dict,axis=1)
len(n)

22629

In [67]:
import pickle
pickle.dump(n, open('14npidict.pkl', 'wb'))

In [68]:
pay_id_dict = d
npi_dict = n

In [73]:
npi_pay_dict = {}
def combine_npi_payid(d,n):
    """
    Input:
        d - Dict of pay_ids as keys with dict as value containing address, fn, ln, specialty, zip5 as keys 
                & all possible alts. for that pay_id as values
        n - Dict of npis as keys with dict as value containing address, fn, ln, specialty, zip5 as keys 
                & all possible alts. for that npi as values 
    Output: 
        None
    """
    for pay_id in d:
        for npi in n:
            t_count = 0
            for val in d[pay_id]:
                if len(d[pay_id][val].intersection(n[npi][val])) >0:
                    t_count += 1
            if t_count >= 4:
                npi_pay_dict[pay_id] = npi

In [74]:
combine_npi_payid(d,n)

In [75]:
len(npi_pay_dict)

13179

In [77]:
len(pay_id_dict)

24267

In [84]:
c = {k:v for k,v in pay_id_dict.items() if k not in npi_pay_dict}
#All the open payment IDs that weren't transfered

In [79]:
len(c)

11088

In [76]:
pickle.dump(npi_pay_dict, open('14_linked.pkl', 'wb'))

In [57]:
npi_pay_dict_13 = pickle.load(open('13_linked.pkl','rb'))

In [59]:
npi_pay_dict_14 = pickle.load(open('14_linked.pkl','rb'))

In [60]:
dif_13_14 = {k:v for k,v in npi_pay_dict_14.items() if k not in npi_pay_dict_13}
#All the NPIs added between 14 & 13

In [61]:
len(dif_13_14), len(npi_pay_dict_13)

(4160, 10345)

In [87]:
len(dif_13_14) + len(npi_pay_dict_13)

14505

In [62]:
npi_pay_dict_full = {**npi_pay_dict_13, **dif_13_14}

In [63]:
len(npi_pay_dict_full)

14505

In [64]:
pickle.dump(npi_pay_dict_full, open('full_linked.pkl', 'wb'))

**npi_dict = n**

**pay_id_dict = d**

### <font color='red'>pay_id_dict</font> = payment ids & all the names & shit

### <font color='blue'>npi_dict</font> = NPI & all the names & shit

# Doing same stuff on the 2015 data

In [92]:
#Then I pick only the columns I want & load the entire csv
pay15 = pd.read_csv('/Volumes/Seagate/Galvanize/2015 Open Payments/OP_DTL_GNRL_PGYR2015_P01172018.csv', \
                    usecols=['Physician_Profile_ID','Physician_First_Name','Physician_Middle_Name', \
                             'Physician_Last_Name','Recipient_State','Total_Amount_of_Payment_USDollars', \
                            'Recipient_Primary_Business_Street_Address_Line1', 'Physician_Specialty', 'Recipient_Zip_Code'])
#Grabbing only the NJ practioners
p15 = pay15[pay15['Recipient_State'].isin(['NJ'])]

/Users/kunal/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
p15.to_csv('/Volumes/Seagate/Galvanize/nj_payments_2015.csv')

In [2]:
p15 = pd.read_csv('/Volumes/Seagate/Galvanize/nj_payments_2015.csv',dtype={'Recipient_Zip_Code':object})

In [3]:
p15.drop('Unnamed: 0',axis=1,inplace=True)

In [30]:
p15.columns

Index(['id', 'fn', 'mn', 'ln', 'address1', 'state', 'zip', 'specialty',
       'amount', 'zip5'],
      dtype='object')

In [31]:
l = p15.columns
p15.rename(columns={l[0]:'id',l[1]:'fn',l[2]:'mn',l[3]:'ln',l[4]:'address1',l[5]:'state',l[6]:'zip',l[-2]:'specialty',l[-1]:'amount'},inplace=True)
#Cleaning the first & last name (removing space & symbols) and the address (removing any symbols)
p15['ln'] = p15['ln'].map(lambda x: str(x).replace(' ',''))
p15['fn'] = p15['fn'].map(lambda x: str(x).replace(' ',''))
p15['mn'] = p15['mn'].map(lambda x: str(x).replace(' ',''))
p15['ln'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p15['ln']]
p15['fn'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p15['fn']]
p15['mn'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p15['mn']]
p15['address1'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p15['address1']]
#Cleaning the zip codes (removing dashes & making sure they are not missing the leading 0 that most NJ zip codes have)
p15['zip'] = [x.replace('-','') for x in p15['zip']]
p15['zip'] = [('0'+str(x)) if len(str(x))==8 or len(str(x))==4 else str(x) for x in p15['zip']]
#Making a column for the 5 digit zip
p15['zip5'] = p15['zip'].map(lambda x: x[:5])

In [ ]:
payids = p15.groupby(['id','fn','ln','mn','zip5','specialty','address1']).count()['amount']

In [7]:
payids = payids.to_frame()

In [8]:
payids.reset_index(inplace=True)

In [78]:
#Dropping amount column because it's not necessary rn
payids.drop('amount',axis=1,inplace=True)
#Finding all the mispelled names/addresses/etc.
mis = payids[payids['id'].isin(payids['id'][payids['id'].duplicated()])].sort_values("id")
#Saving as a CSV for future use
payids.to_csv('/Volumes/Seagate/Galvanize/nj_15_payment_ids.csv')
mis.to_csv('/Volumes/Seagate/Galvanize/nj_15_multi_name_payment_ids.csv')

In [10]:
#This function doesn't include middle name because it was giving me toruble
d = {}
def payment_id(x):
    if x['id'] in d:
        d[x['id']]['fn'].add(x['fn'])
        d[x['id']]['ln'].add(x['ln'])
        d[x['id']]['zip5'].add(x['zip5'])
        d[x['id']]['address1'].add(x['address1'])
        d[x['id']]['specialty'].update(list(x['specialty'].split('|')))
    elif x['id'] not in d:
        d[x['id']] = {'fn':set([x['fn']]), \
                     'ln':set([x['ln']]), \
                     'zip5':set([x['zip5']]), \
                     'address1':set([x['address1']]), \
                     'specialty':set(list(x['specialty'].split('|')))}

In [11]:
payids.head()

,id,fn,ln,mn,zip5,specialty,address1
0,11.0,ALEXANDER,GOTESMAN,NAN,08690,Allopathic & Osteopathic Physicians|Urology,1374 WHITEHORSE HAMILTON SQUARE RD
1,11.0,ALEXANDER,GOTESMAN,NAN,08690,Allopathic & Osteopathic Physicians|Urology,1374 WHITEHORSE HAMILTON SQUARE ROAD
2,11.0,ALEXANDER,GOTESMAN,NAN,08690,Allopathic & Osteopathic Physicians|Urology,1374 WHITEHORSE HAMILTON SQUARE ROAD SUITE 101
3,11.0,ALEXANDER,GOTESMAN,NAN,08690,Allopathic & Osteopathic Physicians|Urology,1374 WHITEHRSE HAM RD STE 202
4,86.0,ANDREW,NAJOVITS,JOSEPH,07450,Allopathic & Osteopathic Physicians|Internal M...,104 CHESTNUT ST


In [12]:
payids.apply(payment_id,axis=1)
len(d)

24043

In [13]:
pickle.dump(d, open('15pay15dict.pkl', 'wb'))

In [15]:
#First I load only a small subset of the data to get an idea of the columns I'll need to use for joining
pres15 = pd.read_csv('/Volumes/Seagate/Galvanize/Prescriptions 2015.csv', nrows=10)
#Then I pick only the columns I want & load the entire csv
scripts15 = pd.read_csv('/Volumes/Seagate/Galvanize/Prescriptions 2015.csv',usecols=list(pres15.columns[:6]))
#Grabbing only the NJ practioners
scripts15nj = scripts15[scripts15['nppes_provider_state'].isin(['NJ'])]
#Renaming the columns so they're easier to call
l = scripts15nj.columns
scripts13nj.rename(columns={l[0]:'npi',l[1]:'ln',l[2]:'fn',l[3]:'city',l[4]:'state',l[5]:'specialty'},inplace=True)

/Users/kunal/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [34]:
#Upper casing the first name, last name, and city
scripts13nj['ln'] = [x.upper() for x in scripts13nj['ln']]
scripts13nj['fn'] = [str(x).upper() for x in scripts13nj['fn']]
scripts13nj['city'] = [x.upper() for x in scripts13nj['city']]
scripts13nj.to_csv('/Volumes/Seagate/Galvanize/2015_scriptsnj.csv')

/Users/kunal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kunal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/kunal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [35]:
#Grouping so I don't have a bunch of duplicate NPIs & I can look for mispellings
s13 = scripts13nj.groupby(['npi','ln','fn','city','specialty']).count()['state'].to_frame()
#Resetting the index becuase all my groupbys are now the index, which wont work for a merge
s13.reset_index(inplace=True)
#Checking if any of the people's names are misspelled
dup13 = s13[s13['npi'].isin(s13['npi'][s13['npi'].duplicated()])].sort_values("npi")

In [36]:
dup13

,npi,ln,fn,city,specialty,state


In [37]:
npi = pd.read_csv('/Volumes/Seagate/Galvanize/nj_idv_npi_may.csv',dtype={'zip':object})

In [38]:
npi.drop('Unnamed: 0',axis=1,inplace=True)
npi.head()

,NPI,Entity Type Code,ln,fn,mn,address1,state,zip
0,1477556421,1.0,ROTH,STEVEN,S.,1107 MANTUA PIKE,NJ,080511606
1,1508869553,1.0,VAN HOUTEN-SAUTER,LEE,ANN,220 PINE ST,NJ,080941137
2,1285637223,1.0,LOWRY,STEVEN,MICHAEL,220 PINE ST,NJ,080941137
3,1225031370,1.0,SY,NENA,L,350 BOULEVARD,NJ,070552840
4,1376546325,1.0,GIULIANO,MICHAEL,GERARD,622 FRANKLIN AVE,NJ,071101247


In [39]:
len(npi), len(npi.loc[npi['Entity Type Code']==1.0])

(102889, 102889)

In [40]:
npi.drop('Entity Type Code',axis=1,inplace=True)

In [41]:
script_npi = s13.merge(npi, left_on='npi',right_on='NPI')
#Dropping unecessary column & the NPI column because it's repeated
script_npi.drop(['NPI','state_x','state_y'],axis=1,inplace=True)

In [42]:
#Cleaning the joined dataframe, removing essentially all punctuation & spaces from the first or last name
script_npi['ln_x'] = script_npi['ln_x'].map(lambda x: x.replace(' ',''))
script_npi['ln_y'] = script_npi['ln_y'].map(lambda x: x.replace(' ',''))
script_npi['fn_x'] = script_npi['fn_x'].map(lambda x: x.replace(' ',''))
script_npi['fn_y'] = script_npi['fn_y'].map(lambda x: x.replace(' ',''))
script_npi['mn'] = script_npi['mn'].map(lambda x: str(x).replace(' ',''))
script_npi['ln_x'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['ln_x']]
script_npi['ln_y'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['ln_y']]
script_npi['fn_x'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['fn_x']]
script_npi['fn_y'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['fn_y']]
script_npi['mn'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['mn']]
script_npi['address1'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['address1']]
#Cleaning the zip codes, NJ has a leading 0 in the zip so it might of been dropped
script_npi['zip'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['zip']]
script_npi['zip'] = [('0'+str(x)) if len(str(x))==8 or len(str(x))==4 else str(x) for x in script_npi['zip']]

In [43]:
#Making a column for the 5 digit zip
script_npi['zip5'] = script_npi['zip'].map(lambda x: x[:5])
"Dealing with mismatched/mispelled names"
#This just lets me see names where the first name & last name don't match up
script_npi.loc[(script_npi['ln_x']!=script_npi['ln_y']) & (script_npi['fn_x']!=script_npi['fn_y'])]

,npi,ln_x,fn_x,city,specialty,ln_y,fn_y,mn,address1,zip,zip5


In [45]:
mis_name = script_npi.loc[(script_npi['ln_x']!=script_npi['ln_y']) | (script_npi['fn_x']!=script_npi['fn_y'])]
mis_name.head(10)

,npi,ln_x,fn_x,city,specialty,ln_y,fn_y,mn,address1,zip,zip5
535,1023053402,RAI,JITHA,LUMBERTON,Gastroenterology,RAIPATEL,JITHA,NAN,693 MAIN ST STE 2,080485043,08048
806,1033219142,MELTON,LAURA,BELVIDERE,Nurse Practitioner,BAYLOR,LAURA,M,540 COUNTY ROAD 519,078232672,07823
1098,1043453616,MANSSON,SARAHJANE,BROWNS MILLS,Internal Medicine,MANSSON,SARAH,NAN,1200 E RIDGEWOOD AVE,074503957,07450
1364,1053640888,SULLIVAN,ERIN,ATLANTIC CITY,Nurse Practitioner,CARUSO,ERIN,MARIE,435 HURFFVILLE CROSS KEYS RD,080122453,08012
1848,1073954251,BRILL,CAREN,MATAWAN,Optometry,MULLNER,CAREN,ELIZABETH,935 ROUTE 34,077473283,07747
1850,1083015762,ARTETA,STEPHANIE,JERSEY CITY,Nurse Practitioner,CORPRON,STEPHANIE,NAN,46 ESSEX ST,073024508,07302
2095,1093009821,MATIAS,VICMARY,CAMDEN,Dentist,KRITHARIS,VICMARY,A,105 ELMORA AVE,072021614,07202
2109,1093092488,FITCH,KIMBERLY,LAKEWOOD,Nurse Practitioner,HERLIHYRAND,KIMBERLY,A,3350 HIGHWAY 138 STE 122,077199694,07719
2333,1104015544,PHILLIPS,ANNE,FLEMINGTON,Psychiatry & Neurology,ROCHFORD,ANNE,THERESE,6 MINNEAKONING RD,088225728,08822
2986,1124235023,CONNOLLY,MARY,CLIFTON,Physician Assistant,REIMUNDO,MARY,ELIZABETH,999 CLIFTON AVE,070132711,07013


In [46]:
#Saving these as CSVs for future use
script_npi.to_csv('/Volumes/Seagate/Galvanize/nj_14_scrip_npi.csv')
mis_name.to_csv('/Volumes/Seagate/Galvanize/nj_14_multi_name_npi.csv')

In [47]:
#This function doesn't include middle name because it was giving me toruble
n = {}
def script_npi_dict(x):
    if x['npi'] in n:
        n[x['npi']]['fn'].update([x['fn_x'],x['fn_y']])
        n[x['npi']]['ln'].update([x['ln_y'],x['ln_y']])
        n[x['npi']]['zip5'].add(x['zip5'])
        n[x['npi']]['address1'].add(x['address1'])
        n[x['npi']]['specialty'].add(x['specialty'])
    elif x['npi'] not in n:
        n[x['npi']] = {'fn':set([x['fn_x'],x['fn_y']]), \
                     'ln':set([x['ln_x'],x['ln_y']]), \
                     'zip5':set([x['zip5']]), \
                     'address1':set([x['address1']]), \
                     'specialty':set([x['specialty']])}

In [48]:
script_npi.head()

,npi,ln_x,fn_x,city,specialty,ln_y,fn_y,mn,address1,zip,zip5
0,1003002072,GOTESMAN,ALEXANDER,HAMILTON,Urology,GOTESMAN,ALEXANDER,NAN,1374 WHITEHORSE HAMILTON SQUARE RD,086903701,08690
1,1003002858,GIBSON,AMY,PLAINFIELD,Nurse Practitioner,GIBSON,AMY,R,1314 PARK AVE,070603253,07060
2,1003011891,SPIRA,ETAN,BELLEVILLE,Gastroenterology,SPIRA,ETAN,B,5 FRANKLIN AVE,071093532,07109
3,1003012410,TINDOC,LORELANE,CLIFTON,Family Practice,TINDOC,LORELANE,PAGULAYAN,780 ALLWOOD RD,070121923,07012
4,1003019910,MIN,JONATHAN,MATAWAN,Dentist,MIN,JONATHAN,NAN,39 CAMBRIDGE DR,077472227,07747


In [49]:
script_npi.apply(script_npi_dict,axis=1)
len(n)

23493

In [50]:
import pickle
pickle.dump(n, open('15npidict.pkl', 'wb'))

In [51]:
pay_id_dict = d
npi_dict = n

In [52]:
npi_pay_dict = {}
def combine_npi_payid(d,n):
    """
    Input:
        d - Dict of pay_ids as keys with dict as value containing address, fn, ln, specialty, zip5 as keys 
                & all possible alts. for that pay_id as values
        n - Dict of npis as keys with dict as value containing address, fn, ln, specialty, zip5 as keys 
                & all possible alts. for that npi as values 
    Output: 
        None
    """
    for pay_id in d:
        for npi in n:
            t_count = 0
            for val in d[pay_id]:
                if len(d[pay_id][val].intersection(n[npi][val])) >0:
                    t_count += 1
            if t_count >= 4:
                npi_pay_dict[pay_id] = npi

In [53]:
combine_npi_payid(d,n)

In [54]:
len(npi_pay_dict)

13661

In [55]:
len(pay_id_dict)

24043

In [65]:
c15 = {k:v for k,v in pay_id_dict.items() if k not in npi_pay_dict}
#All the open payment IDs that weren't transfered

In [66]:
len(c15)

10382

In [67]:
pickle.dump(npi_pay_dict, open('15_linked.pkl', 'wb'))

In [68]:
npi_pay_dict_full = pickle.load(open('full_linked.pkl', 'rb'))

In [69]:
npi_pay_dict_15 = npi_pay_dict

In [70]:
dif_15_13_14 = {k:v for k,v in npi_pay_dict_15.items() if k not in npi_pay_dict_full}
#NPIs from 15 not in 13 or 14

In [71]:
len(dif_15_13_14), len(npi_pay_dict_full)

(2383, 14505)

In [72]:
len(dif_15_13_14) + len(npi_pay_dict_full)

16888

In [73]:
npi_pay_dict_all = {**npi_pay_dict_full, **dif_15_13_14}

In [74]:
len(npi_pay_dict_all)

16888

In [75]:
pickle.dump(npi_pay_dict_all, open('full_linked.pkl', 'wb'))

**npi_dict = n**

**pay_id_dict = d**

### <font color='red'>pay_id_dict</font> = payment ids & all the names & shit

### <font color='blue'>npi_dict</font> = NPI & all the names & shit

## Skip this

In [2]:
pay13 = pd.read_csv('/Volumes/Seagate/Galvanize/2013 Open Payments/OP_DTL_GNRL_PGYR2013_P01172018.csv', \
                    usecols=['Physician_Profile_ID','Physician_First_Name','Physician_Middle_Name', \
                             'Physician_Last_Name','Recipient_State','Total_Amount_of_Payment_USDollars', \
                            'Recipient_Primary_Business_Street_Address_Line1', 'Physician_Specialty', 'Recipient_Zip_Code'])

/Users/kunal/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pay13[:1].to_dict('index')

{0: {'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country': 'United States',
  'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID': 100000000088,
  'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name': 'ELI LILLY AND COMPANY',
  'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State': 'IN',
  'Change_Type': 'UNCHANGED',
  'Charity_Indicator': 'No',
  'City_of_Travel': 'CHICAGO',
  'Contextual_Information': 'GROUND TRANSPORTATION EXPENSES FOR A RESEARCH RELATED MEETING WHICH MAY HAVE LASTED MULTIPLE DAYS',
  'Country_of_Travel': 'United States',
  'Covered_Recipient_Type': 'Covered Recipient Physician',
  'Date_of_Payment': '09/28/2013',
  'Delay_in_Publication_Indicator': 'No',
  'Dispute_Status_for_Publication': 'No',
  'Form_of_Payment_or_Transfer_of_Value': 'In-kind items and services',
  'NDC_of_Associated_Covered_Drug_or_Biological1': 2446430.0,
  'NDC_of_Associated_Covered_Drug_or_Biological2': nan,
  'NDC_of_Associated_Covered_Drug_or_B

In [22]:
pay13.head(2)

,Physician_Profile_ID,Physician_First_Name,Physician_Middle_Name,Physician_Last_Name,Recipient_Primary_Business_Street_Address_Line1,Recipient_State,Physician_Specialty,Total_Amount_of_Payment_USDollars
0,99036.0,CLAUS,G,ROEHRBORN,5323 HARRY HINES BOULEVARD,TX,Allopathic & Osteopathic Physicians|Urology,175.63
1,133635.0,DONALD,CHURCH,BALFOUR,2001 4TH AVE,CA,Allopathic & Osteopathic Physicians|Internal M...,495.00


In [20]:
l = list(p13.columns)
l

['Physician_Profile_ID',
 'Physician_First_Name',
 'Physician_Middle_Name',
 'Physician_Last_Name',
 'Recipient_Primary_Business_Street_Address_Line1',
 'Recipient_State',
 'Recipient_Zip_Code',
 'Physician_Specialty',
 'Total_Amount_of_Payment_USDollars']

In [26]:
p13.rename(columns={l[0]:'id',l[1]:'fn',l[2]:'mn',l[3]:'ln',l[4]:'address1',l[5]:'state',l[6]:'zip',l[-2]:'specialty',l[-1]:'amount'},inplace=True)

In [27]:
p13.head(2)

,id,fn,mn,ln,address1,state,zip,specialty,amount
0,41273.0,MANU,V,CHAKRAVARTHY,1 RWJ PLACE,NJ,8901,Allopathic & Osteopathic Physicians|Internal M...,80.5
1,41273.0,MANU,V,CHAKRAVARTHY,1 RWJ PLACE,NJ,8901,Allopathic & Osteopathic Physicians|Internal M...,58.0


In [3]:
pay13nj = pay13[pay13['Recipient_State'].isin(['NJ'])]

In [4]:
pay13nj.to_csv('/Volumes/Seagate/Galvanize/nj_payments_2013.csv')

In [3]:
pay16 = pd.read_csv('/Volumes/Seagate/Galvanize/2016 Open Payments/OP_DTL_GNRL_PGYR2016_P01172018.csv', \
                    usecols=['Physician_Profile_ID','Physician_First_Name','Physician_Middle_Name', \
                             'Physician_Last_Name','Recipient_State','Total_Amount_of_Payment_USDollars', \
                            'Recipient_Primary_Business_Street_Address_Line1', 'Physician_Specialty', 'Recipient_Zip_Code'], \
                            dtype={'Recipient_Zip_Code':object})
#Grabbing only the NJ practioners
p16 = pay16[pay16['Recipient_State'].isin(['NJ'])]
"""Saving as a CSV for future use"""
p16.to_csv('/Volumes/Seagate/Galvanize/nj_payments_2016.csv')


/Users/kunal/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
l

Index(['Unnamed: 0', 'Physician_Profile_ID', 'Physician_First_Name',
       'Physician_Middle_Name', 'Physician_Last_Name',
       'Recipient_Primary_Business_Street_Address_Line1', 'Recipient_State',
       'Recipient_Zip_Code', 'Physician_Specialty',
       'Total_Amount_of_Payment_USDollars'],
      dtype='object')

In [1]:
import numpy as np
import pandas as pd
import re
import pickle

In [4]:
p16 = pd.read_csv('/Volumes/Seagate/Galvanize/nj_payments_2016.csv', \
                  dtype={'Recipient_Zip_Code':object,'Physician_Profile_ID':object})
#Renaming the columns so they're easier to call
l = p16.columns[1:]
p16.drop('Unnamed: 0',axis=1,inplace=True)
p16.rename(columns={l[0]:'id',l[1]:'fn',l[2]:'mn',l[3]:'ln',l[4]:'address1',l[5]:'state',l[6]:'zip',l[-2]:'specialty',l[-1]:'amount'},inplace=True)

In [5]:
#Cleaning the first & last name (removing space & symbols) and the address (removing any symbols)
p16['ln'] = p16['ln'].map(lambda x: str(x).replace(' ',''))
p16['fn'] = p16['fn'].map(lambda x: str(x).replace(' ',''))
p16['mn'] = p16['mn'].map(lambda x: str(x).replace(' ',''))
p16['ln'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p16['ln']]
p16['fn'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p16['fn']]
p16['mn'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p16['mn']]
p16['address1'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in p16['address1']]

In [6]:
p16['zip'] = [x.replace('-','') for x in p16['zip']]
p16['zip'] = [('0'+str(x)) if len(str(x))==8 or len(str(x))==4 else str(x) for x in p16['zip']]
#Making a column for the 5 digit zip
p16['zip5'] = p16['zip'].map(lambda x: x[:5])

In [7]:
p16.head(2)

,id,fn,mn,ln,address1,state,zip,specialty,amount,zip5
0,332657.0,DIDIER,A,DEMESMIN,33 CLYDE RD,NJ,08873,Allopathic & Osteopathic Physicians|Pain Medic...,714.2,08873
1,332657.0,DIDIER,A,DEMESMIN,33 CLYDE RD,NJ,08873,Allopathic & Osteopathic Physicians|Pain Medic...,25.0,08873


In [8]:
payids = p16.groupby(['id','fn','ln','mn','zip5','specialty','address1',]).count()['amount'].to_frame().reset_index()

In [9]:
#Dropping amount column because it's not necessary rn
payids.drop('amount',axis=1,inplace=True)

In [10]:
#Finding all the mispelled names/addresses/etc.
mis = payids[payids['id'].isin(payids['id'][payids['id'].duplicated()])].sort_values("id")
#Saving as a CSV for future use
payids.to_csv('/Volumes/Seagate/Galvanize/nj_16_payment_ids.csv')
mis.to_csv('/Volumes/Seagate/Galvanize/nj_16_multi_name_payment_ids.csv')

In [2]:
#First I load only a small subset of the data to get an idea of the columns I'll need to use for joining
prescrip16 = pd.read_csv('/Volumes/Seagate/Galvanize/Prescriptions 2016.csv', nrows=100)

In [3]:
scripts16 = pd.read_csv('/Volumes/Seagate/Galvanize/Prescriptions 2016.csv',usecols=list(prescrip16.columns[:6]))
#Grabbing only the NJ practioners
scripts16nj = scripts16[scripts16['nppes_provider_state'].isin(['NJ'])]
#Renaming the columns so they're easier to call
l = scripts16nj.columns
scripts16nj.rename(columns={l[0]:'npi',l[1]:'ln',l[2]:'fn',l[3]:'city',l[4]:'state',l[5]:'specialty'},inplace=True)

/Users/kunal/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [4]:
scripts16nj['ln'] = [x.upper() for x in scripts16nj['ln']]
scripts16nj['fn'] = [str(x).upper() for x in scripts16nj['fn']]
scripts16nj['city'] = [x.upper() for x in scripts16nj['city']]
"""Saving as a csv for future use"""
scripts16nj.to_csv('/Volumes/Seagate/Galvanize/2016_scriptsnj.csv')

/Users/kunal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/kunal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kunal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [2]:
scripts16nj = pd.read_csv('/Volumes/Seagate/Galvanize/2016_scriptsnj.csv',dtype={'npi':object,'zip':object})

In [3]:
s16 = scripts16nj.groupby(['npi','ln','fn','city','specialty']).count()['state'].to_frame()
#Resetting the index becuase all my groupbys are now the index, which wont work for a merge
s16.reset_index(inplace=True)

In [4]:
dup16 = s16[s16['npi'].isin(s16['npi'][s16['npi'].duplicated()])].sort_values("npi")

In [5]:
dup16

,npi,ln,fn,city,specialty,state


In [16]:
npi_idv = pd.read_csv('/Volumes/Seagate/Galvanize/nj_idv_npi_may.csv', \
                  dtype={'zip':object,'NPI':object})

In [17]:
script_npi = s16.merge(npi_idv, left_on='npi',right_on='NPI')

In [19]:
script_npi.drop(['Unnamed: 0','Entity Type Code','NPI','state_x','state_y'],axis=1,inplace=True)

In [20]:
script_npi.head(2)

,npi,ln_x,fn_x,city,specialty,ln_y,fn_y,mn,address1,zip
0,1003001678,EIRICH,REBECCA,TURNERSVILLE,Nurse Practitioner,EIRICH,REBECCA,NaN,188 FRIES MILL RD,080122015
1,1003002072,GOTESMAN,ALEXANDER,HAMILTON,Urology,GOTESMAN,ALEXANDER,NaN,1374 WHITEHORSE HAMILTON SQUARE RD,086903701


In [21]:
script_npi['ln_x'] = script_npi['ln_x'].map(lambda x: x.replace(' ',''))
script_npi['ln_y'] = script_npi['ln_y'].map(lambda x: x.replace(' ',''))
script_npi['fn_x'] = script_npi['fn_x'].map(lambda x: x.replace(' ',''))
script_npi['fn_y'] = script_npi['fn_y'].map(lambda x: x.replace(' ',''))
script_npi['mn'] = script_npi['mn'].map(lambda x: str(x).replace(' ',''))
script_npi['ln_x'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['ln_x']]
script_npi['ln_y'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['ln_y']]
script_npi['fn_x'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['fn_x']]
script_npi['fn_y'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['fn_y']]
script_npi['mn'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['mn']]
script_npi['address1'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['address1']]
#Cleaning the zip codes, NJ has a leading 0 in the zip so it might of been dropped
script_npi['zip'] = [re.sub(r'[^\w\s]','',str(x).upper()) for x in script_npi['zip']]
script_npi['zip'] = [('0'+str(x)) if len(str(x))==8 or len(str(x))==4 else str(x) for x in script_npi['zip']]

In [22]:
script_npi['zip5'] = script_npi['zip'].map(lambda x: x[:5])
"""Dealing with mismatched/mispelled names"""
#This just lets me see names where the first name & last name don't match up
script_npi.loc[(script_npi['ln_x']!=script_npi['ln_y']) & (script_npi['fn_x']!=script_npi['fn_y'])]

,npi,ln_x,fn_x,city,specialty,ln_y,fn_y,mn,address1,zip,zip5


In [24]:
mis_name.to_csv('/Volumes/Seagate/Galvanize/nj_16_multi_name_npi.csv')

In [37]:
d = {}
def payment_id(x):
    if x['id'] in d:
        d[x['id']]['fn'].add(x['fn'])
        d[x['id']]['ln'].add(x['ln'])
        d[x['id']]['zip5'].add(x['zip5'])
        d[x['id']]['address1'].add(x['address1'])
        d[x['id']]['specialty'].update(list(x['specialty'].split('|')))
    elif x['id'] not in d:
        d[x['id']] = {'fn':set([x['fn']]), \
                     'ln':set([x['ln']]), \
                     'zip5':set([x['zip5']]), \
                     'address1':set([x['address1']]), \
                     'specialty':set(list(x['specialty'].split('|')))}
#Applying the function to our payment_ids
payids.apply(payment_id,axis=1)

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
         ... 
68062    None
68063    None
68064    None
68065    None
68066    None
68067    None
68068    None
68069    None
68070    None
68071    None
68072    None
68073    None
68074    None
68075    None
68076    None
68077    None
68078    None
68079    None
68080    None
68081    None
68082    None
68083    None
68084    None
68085    None
68086    None
68087    None
68088    None
68089    None
68090    None
68091    None
Length: 68092, dtype: object

In [38]:
d

{1000064.0: {'address1': {'1204 FRANCYNE WAY', '1551 ROUTE 138'},
  'fn': {'REBEKAH'},
  'ln': {'VALTHATY'},
  'specialty': {'Allopathic & Osteopathic Physicians',
   'Obstetrics & Gynecology'},
  'zip5': {'07083', '07719'}},
 100010.0: {'address1': {'370 GRAND AVE', '370 GRAND AVENUE'},
  'fn': {'WENDY'},
  'ln': {'HURST'},
  'specialty': {'Allopathic & Osteopathic Physicians',
   'Gynecology',
   'Legal Medicine',
   'Obstetrics & Gynecology',
   'Other Service Providers'},
  'zip5': {'07631'}},
 1000196.0: {'address1': {'71 VALLEY ST'},
  'fn': {'KEVIN'},
  'ln': {'JOHNSON'},
  'specialty': {'Dental Providers', 'Dentist', 'Endodontics'},
  'zip5': {'07079'}},
 1000208.0: {'address1': {'25 PROSPECT AVE',
   '759 HAMBURG TPKE',
   '85 CHESTNUT RIDGE RD'},
  'fn': {'LYNN'},
  'ln': {'CORRIGAN'},
  'specialty': {'Allopathic & Osteopathic Physicians', 'Family Medicine'},
  'zip5': {'07470', '07601', '07645'}},
 100040.0: {'address1': {'47 MAPLE ST', '741 NORTHFIELD AVE'},
  'fn': {'LAURA

In [39]:
pickle.dump(d, open('16paydict.pkl', 'wb'))

In [40]:
n = {}
def script_npi_dict(x):
    if x['npi'] in n:
        n[x['npi']]['fn'].update([x['fn_x'],x['fn_y']])
        n[x['npi']]['ln'].update([x['ln_y'],x['ln_y']])
        n[x['npi']]['zip5'].add(x['zip5'])
        n[x['npi']]['address1'].add(x['address1'])
        n[x['npi']]['specialty'].add(x['specialty'])
    elif x['npi'] not in n:
        n[x['npi']] = {'fn':set([x['fn_x'],x['fn_y']]), \
                     'ln':set([x['ln_x'],x['ln_y']]), \
                     'zip5':set([x['zip5']]), \
                     'address1':set([x['address1']]), \
                     'specialty':set([x['specialty']])}

In [41]:
script_npi.apply(script_npi_dict,axis=1)

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
         ... 
24263    None
24264    None
24265    None
24266    None
24267    None
24268    None
24269    None
24270    None
24271    None
24272    None
24273    None
24274    None
24275    None
24276    None
24277    None
24278    None
24279    None
24280    None
24281    None
24282    None
24283    None
24284    None
24285    None
24286    None
24287    None
24288    None
24289    None
24290    None
24291    None
24292    None
Length: 24293, dtype: object

In [43]:
pickle.dump(n, open('16npidict.pkl', 'wb'))

In [44]:
pay_id_16_dict = d
npi_16_dict = n

In [45]:
npi_pay_dict = {}
def combine_npi_payid(d,n):
    """
    Input:
        d - Dict of pay_ids as keys with dict as value containing address, fn, ln, specialty, zip5 as keys
                & all possible alts. for that pay_id as values
        n - Dict of npis as keys with dict as value containing address, fn, ln, specialty, zip5 as keys
                & all possible alts. for that npi as values
    Output:
        None
    """
    for pay_id in d:
        for npi in n:
            t_count = 0
            for val in d[pay_id]:
                if len(d[pay_id][val].intersection(n[npi][val])) >0:
                    t_count += 1
            if t_count >= 4:
                npi_pay_dict[pay_id] = npi


In [46]:
#Combining them using the function i created
combine_npi_payid(pay_id_16_dict, npi_16_dict)
#Pickling it so I can utilize it with ease later
pickle.dump(npi_pay_dict, open('16_linked.pkl', 'wb'))

In [47]:
print('hi')

hi


In [53]:
npi_pay_dict_all = pickle.load(open('/Volumes/Seagate/Galvanize/Pickles/full_linked.pkl', 'rb'))

In [ ]:
dif_13_14_15_16 = {k:v for k,v in npi_pay_dict_16.items() if k not in npi_pay_dict_13_14_15}
npi_pay_dict_all = {**npi_pay_dict_all, **dif_13_14_15_16}

In [54]:
dif = {k:v for k,v in npi_pay_dict.items() if k not in npi_pay_dict_all}

In [56]:
len(dif)

1829

In [57]:
npi_pay_dict_all = {**npi_pay_dict_all, **dif_13_14_15_16}

In [58]:
pickle.dump(npi_pay_dict_all, open('full_linked.pkl', 'wb'))